In [14]:
import sqlalchemy as sa
import pandas as pd
engine = sa.create_engine("sqlite:///myhome.db")
sql = sa.text("SELECT * FROM p1g")
df = pd.read_sql(sql, con=engine)
df.dropna(axis=0, inplace=True)
df.head()


,time,Total gas used
7353,1647612900,3422.015
7354,1647613800,3422.028
7355,1647614700,3422.033
7356,1647615600,3422.065
7357,1647616500,3422.089


In [15]:
import seaborn as sns
sns.set_theme(style="ticks")

# 
# Define the palette as a list to specify exact values
palette = sns.color_palette("rocket_r")

# Plot the lines on two facets
sns.relplot(
    data=df,
    x="time", y="Total gas used",
    kind="line", palette=palette,
    height=5, aspect=.75, facet_kws=dict(sharex=False)
)

C:\Users\Alisa_Wang\AppData\Local\Temp\ipykernel_8764\2602075672.py:9: UserWarning: Ignoring `palette` because no `hue` variable has been assigned.
  sns.relplot(
